In [1]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
from datasets import load_dataset
import pandas as pd

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  2


In [3]:
strategy = tf.distribute.MirroredStrategy()

In [4]:
train=pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test=pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

In [5]:
x_train=train["text"]
x_test=test["text"]


In [6]:
import re
x_train=x_train.apply(lambda x: re.sub(r'http\S+|www\S+', '', x))

# Remove Hashtags
x_train =x_train.apply(lambda x: re.sub(r'#\w+', '', x))

# Remove Mentions
x_train =x_train.apply(lambda x: re.sub(r'@\w+', '', x))

# Remove extra spaces after cleaning
x_train =x_train.apply(lambda x: ' '.join(x.split()))

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(x_train, train["target"], test_size=0.2, random_state=42)

In [8]:
y_val

2644    1
2227    0
5448    1
132     0
6845    0
       ..
1835    0
506     1
3592    1
6740    1
1634    0
Name: target, Length: 1523, dtype: int64

In [9]:
import keras_nlp
from transformers import BertTokenizer, TFBertModel
import keras

with strategy.scope():
    preset= "distil_bert_base_en_uncased"
    
    preprocessor = keras_nlp.models.DistilBertPreprocessor.from_preset(preset,
                                                                   sequence_length=160,
                                                                   name="preprocessor_4_tweets"
                                                                  )

    classifier = keras_nlp.models.DistilBertClassifier.from_preset(preset,
                                                               preprocessor = preprocessor, 
                                                               num_classes=2)

    classifier.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(1e-5),
    metrics=['accuracy']
    )

    
    
classifier.summary()

Preprocessor: "preprocessor_4_tweets"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ distil_bert_tokenizer (DistilBertTokenizer)        │                                              30,522 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "distil_bert_classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ distil_bert_backbone          │ (None, None, 768)         │      66,362,880 │ padding_mask[0][0],        │
│ (DistilBertBackbone)          │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ get_item (GetItem)            │ (None, 768)               │               0 │ distil_bert_backbone[0][0] │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pooled_dense (Dense)          │ (None, 768)               │         590,592 │ get_item[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ output_dropout (Dropout)      │ (None, 768)               │               0 │ pooled_dense[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ logits (Dense)                │ (None, 2)                 │           1,538 │ output_dropout[0][0]       │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 66,955,010 (255.41 MB)

 Trainable params: 66,955,010 (255.41 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:

classifier.fit(x=X_train, y=y_train,validation_data=(X_val, y_val),epochs=3)
# Re-compile (e.g., with a new learning rate).


Epoch 1/3
191/191 ━━━━━━━━━━━━━━━━━━━━ 140s 422ms/step - accuracy: 0.6799 - loss: 0.6044 - val_accuracy: 0.8255 - val_loss: 0.4164
Epoch 2/3
191/191 ━━━━━━━━━━━━━━━━━━━━ 81s 422ms/step - accuracy: 0.8262 - loss: 0.4022 - val_accuracy: 0.8281 - val_loss: 0.4013
Epoch 3/3
191/191 ━━━━━━━━━━━━━━━━━━━━ 81s 422ms/step - accuracy: 0.8717 - loss: 0.3290 - val_accuracy: 0.8150 - val_loss: 0.4217


In [11]:
import re
x_test=x_test.apply(lambda x: re.sub(r'http\S+|www\S+', '', x))

# Remove Hashtags
x_test =x_test.apply(lambda x: re.sub(r'#\w+', '', x))

# Remove Mentions
x_test =x_test.apply(lambda x: re.sub(r'@\w+', '', x))

# Remove extra spaces after cleaning
x_test =x_test.apply(lambda x: ' '.join(x.split()))

In [12]:
prediction=classifier.predict(x=test["text"], batch_size=2)

1632/1632 ━━━━━━━━━━━━━━━━━━━━ 25s 14ms/step


In [13]:
prediction.shape

(3263, 2)

In [14]:
import numpy as np
sample_submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
sample_submission["target"] = np.argmax(prediction, axis=1)

In [15]:
sample_submission

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [16]:

sample_submission.to_csv("submission.csv", index=False)